In [51]:
import json
import pandas as pd
import numpy as np
import random
import sys

In [52]:
sys.setrecursionlimit(10**6)

data = np.array(pd.read_csv('test2.csv'),dtype= int)
T,L = data.shape[0],data.shape[1]

M,A,E =0,0,0

In [53]:
def varSelection(array,N,D,j):

    if j==D:
        return 0,j

    l =0
    unassigned = []
    for l in range(0,N):
        if(array[l][j]==0):
            unassigned.append(l)
    
    if len(unassigned) == 0:
        return varSelection(array,N,D,j+1)

    for k in unassigned:
        if j>0 and (array[k][j-1] == 1 or array[k][j-1]==3): #if previous day morning or evening
            return k,j


    return random.choice(unassigned),j

In [54]:
def valueSelection(array,N,D,i,j,m,a,e,r,availableVal,preferedVal,nurseRest):

    if j>=1 and (array[i][j-1] == 1 or array[i][j-1]==3):   #if last day morning or even

        if nurseRest[i]==0 and availableVal[4]==1 and r!=0:
            return 4 

        elif availableVal[3]==1 and e!=0:
            return 3            #atlast evening
    
        elif availableVal[2]==1 and a!=0:
            return 2            #Giving Afternoon
        
        
        elif availableVal[4]==1 and r!=0:
            return 4

        
        else:
            return -1
    

    if nurseRest[i]==0 and availableVal[4]==1 and r!=0:
        return 4 

    
    elif availableVal[3]==1 and e!=0:
        return 3            #atlast evening

    elif availableVal[2]==1 and a!=0:
        return 2            #Giving Afternoon
    
    elif availableVal[1]==1 and m!=0:
        return 1

    elif availableVal[4]==1 and r!=0:
        return 4

    return -1

In [55]:
def checkValid(array,i,j,val,m,a,e,r,N):

    if val == 1 and (m ==0 or(j>=1 and  (array[i][j-1]==1 or array[i][j-1]==3))):
        return False

    if val ==2 and a==0:
        return False

    if val==3 and e ==0:
        return False
        
    if val ==4 and r==0:
        return False
    
    if i==N-1:
        if val==1 and (a!=0 or e!=0):
            return False
        elif val==2 and (m!=0 or e!=0):
            return False
        elif val==3 and (a!=0 or m!=0):
            return False
    return True

In [56]:
def preferVal(m,a,e):
    
    d = {1:m,2:a,3:e}

    sortedDict = sorted(d.items(),key=lambda x:x[1],reverse=True)
    most = 0
    for i in sortedDict:
        return(i[0])


In [57]:
def NurseSol(array,N,D,m,a,e,r,nurseRest):
    
    z=0
    for _ in range(0,m):
        array[z][0] =1
        z+=1
    for _ in range(0,a):
        array[z][0]=2
        z+=1
    for _ in range(0,e):
        array[z][0]=3
        z+=1
    for _ in range(0,r):
        array[z][0]=4
        nurseRest[z]+=1
        z+=1
    sol = nurseSolver(array,0,1,N,D,m,a,e,r,nurseRest)
    if sol:
        return array
    return np.zeros((N,D))

In [58]:
def validfurther(array,j,m,a,e,N):
    
    if j==0:
        return True

    count = 0
    total = 0
    for k in range(0,N):
        if array[k][j]==0:
            total+=1
            if array[k][j-1] == 1 or array[k][j-1] == 3:
                count+=1
    
    if total-count < m:
        return False
    
    return True

In [59]:
def nurseSolver(array,i,j,N,D,m,a,e,r,nurseRest):
    print (i,j)
    if(j==D):
        return True
    
    if not validfurther(array,j,m,a,e,N):
        return False

    valAvailable = [1,1,1,1,1]
    
    for x in range(0,4):

        y = preferVal(m,a,e)    #which is most remaining
        
        value = valueSelection(array,N,D,i,j,m,a,e,r,valAvailable,y,nurseRest)

        if(value==-1):      
            return False
        
        valAvailable[value]=0

        if checkValid(array,i,j,value,m,a,e,r,N):
            result = False

            array[i][j] =value

            if value ==1:
                m-=1
            elif value ==2:
                a-=1
            elif value==3:
                e-=1 
            elif value ==4:
                r-=1
                nurseRest[i]+=1
            
            if i == N-1:
                if(j%7==6):
                    nurseRest1=np.zeros(N)
                    result = nurseSolver(array,0,j+1,N,D,M,A,E,R,nurseRest1)
                else:
                    result = nurseSolver(array,0,j+1,N,D,M,A,E,R,nurseRest)
            else:
                 result = nurseSolver(array,i+1,j,N,D,m,a,e,r,nurseRest)
                 
            
            if result:
                return result
            else:
                array[i][j]=0
                if value ==1:
                    m+=1
                elif value ==2:
                    a+=1
                elif value==3:
                    e+=1
                elif value==4:
                    r+=1
                    nurseRest[i]-=1
    return False
        

In [60]:

if(L==5):
    for i in range (0,T):
        N,D,M,A,E = data[i]
        R = N-M-A-E
        array = np.zeros((N,D))

        if(M+A+E> 6*N/7) or (N < 2*M+E ):
            print("NO SOLUTION")
            continue
        
        nurseRest = np.zeros(N)
        solu = NurseSol(array,N,D,M,A,E,R,nurseRest)
        
        soln_list = []
        
        for p in range (0,D):
            dic = {}
            for q  in range(0,N):
                k = 'N'+str(q)+'_'+str(p)
                if(solu[q][p]==1):
                    dic[k]='M'
                elif(solu[q][p]==2):
                    dic[k]='A'
                elif(solu[q][p]==3):
                    dic[k]='E'
                else:
                    dic[k]='R'
            soln_list.append(dic)
        
        with open('solution.json','w') as file:
            for d in soln_list:
                json.dump(d,file)
                file.write("\n")

0 1
1 1
2 1
3 1
4 1
5 1
6 1
7 1
8 1
9 1
10 1
11 1
12 1
13 1
14 1
14 1
14 1
15 1
15 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1
29 1
0 2
1 2
2 2
3 2
4 2
4 2
4 2
5 2
5 2
5 2
6 2
6 2
6 2
7 2
7 2
7 2
8 2
8 2
8 2
9 2
10 2
11 2
12 2
13 2
14 2
15 2
16 2
17 2
18 2
18 2
18 2
19 2
19 2
19 2
20 2
20 2
20 2
21 2
21 2
21 2
22 2
22 2
22 2
23 2
24 2
25 2
26 2
27 2
28 2
29 2
0 3
1 3
2 3
3 3
4 3
5 3
6 3
7 3
8 3
9 3
10 3
11 3
12 3
12 3
13 3
13 3
14 3
14 3
15 3
15 3
16 3
16 3
17 3
18 3
19 3
20 3
21 3
22 3
23 3
24 3
25 3
26 3
27 3
28 3
29 3
0 4
1 4
2 4
3 4
4 4
5 4
6 4
7 4
8 4
9 4
10 4
11 4
11 4
12 4
13 4
14 4
15 4
16 4
17 4
18 4
19 4
20 4
21 4
22 4
23 4
24 4
25 4
26 4
27 4
28 4
29 4
0 5
1 5
2 5
3 5
4 5
5 5
6 5
7 5
8 5
9 5
10 5
11 5
12 5
12 5
13 5
13 5
14 5
14 5
15 5
15 5
16 5
16 5
17 5
18 5
19 5
20 5
21 5
22 5
23 5
24 5
25 5
26 5
27 5
28 5
29 5
0 6
1 6
2 6
3 6
4 6
5 6
6 6
7 6
8 6
9 6
10 6
11 6
11 6
12 6
13 6
14 6
15 6
16 6
17 6
18 6
19 6
20 6
21 6
22 6
23 6
24 6
25 6
26 6
27 6
